In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append('/Users/kevingmagana/DSI/capstone/capstone-update/Capstone_Proposal/py_scripts/')
import find_citation, index
import networkx as nx
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline
from collections import defaultdict
import copy
import re
import boto3
from io import StringIO, BytesIO
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from gensim.models import phrases
from gensim.models.phrases import Phraser
import os
# from _pickle import *
import _pickle as cPickle



## (1) Load Merged Data (with Scraped html formatting )

In [2]:
# 's3' is a key word. create connection to S3 using default config and all buckets within S3
s3 = boto3.resource('s3')
client = boto3.client('s3') #low-level functional API

obj = client.get_object(Bucket='court-case-data', Key='merged_data_with_html_format.csv')
df = pd.read_csv(BytesIO(obj['Body'].read()))



In [3]:
corp = df.dropna()
corp.head()

,Unnamed: 0,case_text,case_title,court,date,docket,tags,type_of_law,url,web_source,html_format
1,1,"United States Court of Appeals,Tenth Circuit.\...",Porro v. Barnes,United States Tenth Circuit,11/09/2010,10-6002,"Civil Rights, Constitutional Law, Immigration ...",case,http://caselaw.findlaw.com/us-10th-circuit/154...,findlaw.com,"[<div class=""caselawcontent searchable-content..."
2,2,"United States Court of Appeals,Tenth Circuit.\...",Garcia-Carbajal v. Holder,United States Tenth Circuit,11/05/2010,09-9558,"Administrative Law, Immigration Law",case,http://caselaw.findlaw.com/us-10th-circuit/154...,findlaw.com,"[<div class=""caselawcontent searchable-content..."
3,3,"United States Court of Appeals,Tenth Circuit.\...",US v. Rendon-Alamo,United States Tenth Circuit,10/19/2010,10-2089,"Sentencing, Immigration Law, Criminal Law & Pr...",case,http://caselaw.findlaw.com/us-10th-circuit/154...,findlaw.com,"[<div class=""caselawcontent searchable-content..."
5,5,"United States Court of Appeals,Tenth Circuit.\...",Iliev v. Holder,United States Tenth Circuit,07/19/2010,09-9517,"Administrative Law, Immigration Law",case,http://caselaw.findlaw.com/us-10th-circuit/153...,findlaw.com,"[<div class=""caselawcontent searchable-content..."
6,6,"United States Court of Appeals,Tenth Circuit.\...",US v. Adame-Orozco,United States Tenth Circuit,06/04/2010,09-3296,"Criminal Law & Procedure, Immigration Law",case,http://caselaw.findlaw.com/us-10th-circuit/152...,findlaw.com,"[<div class=""caselawcontent searchable-content..."


## (2) Data Cleaning, Index, and finally and Bag of Words (TF-IDF)

In [ ]:
"""
Initial Data Pre-Processing 
Steps: 
    (1) Tokenize everything 
    (2) Remove Stop Words
    (3) Stem / Lemm 
    (4) -- Create classes 

Wish List: 
    (1) Bigram, Trigram, and N-Gram tokenizer. 
    (2) Citation Parsing 
"""

In [6]:
def tokenize(tokenize_texts): 
    """
    :param case_text: list of strings (case_texts)
    :returns: list of strings, tokenized word.
    """
    table = str.maketrans({key: None for key in string.punctuation})
    return [word_tokenize(case.translate(table).lower()) for case in tokenize_texts]


def remove_stop_words(remove_stop_text):
    """ 
    :param case_text: list of lists of strings (tokens)
    :returns: list of lists of strings(tokens) without stopwords
    """
    stop = set(stopwords.words('english'))
    return [[word for word in words if word not in stop] for words in remove_stop_text]
     
def stem_words(stem_word_text): 
    """
    :param without_stop: list of lists without stop words
    :return: list of lists of strings (stemmed words)
    """
    snowball = SnowballStemmer('english')
    return [[snowball.stem(word) for word in words] for words in stem_word_text]

### process_cases takes 3-5 mins to run

In [5]:
def process_cases(cases):
    """
    :param cases: dataframe, where column1 is case_titles, column2 is case_texts
    :returns case_to_terms: dict, where keys are case_titles and values are words/ tokens in that case 
        EX: 
            case_to_terms = {case_title: [world1, word2], ...}
    """
    case_to_terms = {}
    tokenized= tokenize(cases['case_text'])
    remove_stops = remove_stop_words(tokenized)
    stem_the_words = stem_words(remove_stops)
    
    for ind, case in enumerate(cases['case_title']):
        case_to_terms[case] = stem_the_words[ind]
    return case_to_terms

In [4]:
# corp['case_title'].describe()

# corp[corp['case_title']=='Singh v. Gonzales']

# corp['title_date'] = corp['case_title'] + corp['date']

corp['title_date'] = corp[['case_title', 'date']].apply(lambda x: ' '.join(x), axis=1)


/Users/kevingmagana/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [5]:
corpus = corp.iloc[:, [1, 11]]
corpus.head()

,case_text,title_date
1,"United States Court of Appeals,Tenth Circuit.\...",Porro v. Barnes 11/09/2010
2,"United States Court of Appeals,Tenth Circuit.\...",Garcia-Carbajal v. Holder 11/05/2010
3,"United States Court of Appeals,Tenth Circuit.\...",US v. Rendon-Alamo 10/19/2010
5,"United States Court of Appeals,Tenth Circuit.\...",Iliev v. Holder 07/19/2010
6,"United States Court of Appeals,Tenth Circuit.\...",US v. Adame-Orozco 06/04/2010


### If you dropna here, you MUST dropna in the S3 CSV bucket. 

In [7]:
# corp = df.dropna()
# corp.head(2)

### Warning -- this takes about 3-5 minutes to run!!!
case_to_terms = process_cases(corp)
len(case_to_terms)


4018

>>> ### Will have to figure out HOW to sync up the case titles from the dataframe here and the titles I used in graphing earlier this week




In [169]:
#input = [word1, word2, ...]
#output = {word1: [pos1, pos2], word2: [pos2, pos434], ...}
def index_one_file(termlist):
    """
    :param termlist: list of terms  
    :returns: dict, single caseIndex
        EX: 
        {word1: [pos1, pos2], word2: [pos2, pos434], ...}
    
    """
    caseIndex={}
    for index, word in enumerate(termlist):
        if word in caseIndex.keys():
            caseIndex[word].append(index)
        else: 
            caseIndex[word].append = [index]
    return caseIndex

#input = {filename: [word1, word2, ...], ...}
#res = {filename: {word: [pos1, pos2, ...]}, ...}
def make_indices(termlists):
    total = {}
    for filename in termlists.keys():
        total[filename] = self.index_one_file(termlists[filename])
    return total


## Run the code

In [85]:

docs = corp[0:3]
docs_names = docs.case_title
doc_texts = docs.case_text

# process_cases(docs_names, doc_texts)

## Wish List: *** Bigram, Trigram, & N-gram *** Tokenizing

### Grab the Citation Parser & Include it here

In [151]:
docs = [word_tokenize(content) for content in doc_texts]
test = tokenize(doc_texts[1:3])
phrases = Phrases(test)
bigram = Phraser(phrases)
# sent = [u'the', u'mayor', u'of', u'new', u'york', u'was', u'there']
# print(list(bigram[test[1]]))
# print()

### RESET INDEX

In [6]:
corpus = corp.iloc[:, [1, 11]]
corpus = corpus.drop_duplicates(subset='title_date')
corpus = corpus.reset_index(drop=True)
corpus.describe()

,case_text,title_date
count,4470,4470
unique,4310,4470
top,"United States Court of Appeals,Fifth Circuit.\...",US v. Ruiz-Lopez 04/25/2014
freq,3,1


In [7]:
corpus.head()

,case_text,title_date
0,"United States Court of Appeals,Tenth Circuit.\...",Porro v. Barnes 11/09/2010
1,"United States Court of Appeals,Tenth Circuit.\...",Garcia-Carbajal v. Holder 11/05/2010
2,"United States Court of Appeals,Tenth Circuit.\...",US v. Rendon-Alamo 10/19/2010
3,"United States Court of Appeals,Tenth Circuit.\...",Iliev v. Holder 07/19/2010
4,"United States Court of Appeals,Tenth Circuit.\...",US v. Adame-Orozco 06/04/2010


## Testing index.py BuildIndex class (5-8 mins to run)

In [8]:
build_index = index.BuildIndex(corpus)


## Save/Pickle the Index Object (2-3 mins)


In [9]:
# from _pickle import *

file_path = "/Users/kevingmagana/DSI/capstone/build_index_object_2.pkl"
n_bytes = 2**31
max_bytes = 2**31 - 1
# data = bytearray(n_bytes)
data = build_index

## write
bytes_out = cPickle.dumps(data)
with open(file_path, 'wb') as f_out:
    for idx in range(0, n_bytes, max_bytes):
        f_out.write(bytes_out[idx:idx+max_bytes])
        


## To load - takes about (2-3 mins)

In [14]:
import os

file_path = "/Users/kevingmagana/DSI/capstone/build_index_object_2.pkl"

## read
bytes_in = bytearray(0)
input_size = os.path.getsize(file_path)
with open(file_path, 'rb') as f_in:
    for _ in range(0, input_size, max_bytes):
        bytes_in += f_in.read(max_bytes)

data2 = cPickle.loads(bytes_in)



In [15]:
data2.case_titles.unique()

array(['Porro v. Barnes 11/09/2010',
       'Garcia-Carbajal v. Holder 11/05/2010',
       'US v. Rendon-Alamo 10/19/2010', ...,
       'Hui v. Castaneda 05/03/2010', 'Padilla v. Kentucky 03/31/2010',
       'Kucana v. Holder 01/20/2010'], dtype=object)

## Load the Index Object

In [181]:
with open("build_index_object.pkl", "rb") as myFile:
    index_object = pickle.load(myFile)



IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

vectorizer = TfidfVectorizer(stop_words='english')
vectors = vectorizer.fit_transform(corpus.case_text).toarray()

In [19]:
## The first index is the row, and second is the column. 
vectors[0]

array([0.        , 0.00481816, 0.        , ..., 0.        , 0.        ,
       0.        ])

In [26]:
df = pd.read_csv('/Users/kevingmagana/DSI/capstone/case_corpus.csv')
df = df.dropna()
df['title_date'] = df[['case_title', 'date']].apply(lambda x: ' '.join(x), axis=1)
df = df.iloc[:, [2, 12]]
df = df.drop_duplicates(subset='title_date')
df = df.reset_index(drop=True)

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4470 entries, 0 to 4469
Data columns (total 2 columns):
case_text     4470 non-null object
title_date    4470 non-null object
dtypes: object(2)
memory usage: 69.9+ KB


In [29]:
answer = [126, 46, 33, 34, 35, 36, 37, 38, 39, 40]


In [31]:
list1 = ["Hey Hey Hey"]
str1 = ''.join(list1)
str1

'Hey Hey Hey'